# Internship Finn Sherry @ Sioux Mathware

---

# Bayesian grey-box system identification for thermal effects: MCMC using Turing
In this notebook, we apply the MCMC using [Turing.jl](https://github.com/TuringLang/Turing.jl) combined with [DifferentialEquations.jl](https://github.com/SciML/DifferentialEquations.jl). The goal is to assess whether MCMC is a method that is usable for the rest of my project. This notebook continues on from the Markov Chain Monte Carlo part of my [main (Julia) notebook](sysid-thermal-AR.ipynb). Turing and DifferentialEquations are completely composable. This means that we can just plug in the differential equations governing the dynamics of our system.

Last update: 27-07-2022

$\renewcommand{\vec}[1]{\boldsymbol{\mathrm{#1}}}$
$\newcommand{\covec}[1]{\hat{\vec{#1}}}$
$\newcommand{\mat}[1]{\boldsymbol{\mathrm{#1}}}$
$\newcommand{\inv}[1]{#1^{-1}}$
$\newcommand{\Expectation}{\mathbb{E}}$
$\newcommand{\Variance}{\mathbb{V}}$

In [ ]:
using Turing, DifferentialEquations, Random, LinearAlgebra, StatsBase # Computational
using Measures, LaTeXStrings, StatsPlots # Formatting
Random.seed!(987654321)

## Physical Model
Here, we first define the differential equations that govern our system. To start off with, we have 3 blocks, with unknown parameters $k_{12}, k_{23}, h_a$, whose temperatures are governed by
$$\frac{d}{dt}\begin{pmatrix} m_1 c_{p, 1} T_1 \\ m_2 c_{p, 2} T_2 \\ m_3 c_{p, 3} T_3 \end{pmatrix} = 
\begin{pmatrix} -k_{12} & k_{12} & 0 \\ k_{12} & -(k_{12} + k_{23}) & k_{23} \\ 0 & k_{23} & -k_{23} \end{pmatrix} \begin{pmatrix} T_1 \\ T_2 \\ T_3 \end{pmatrix} + \begin{pmatrix} \Phi_1 \\ \Phi_2 \\ \Phi_3 \end{pmatrix} +
h_a \begin{pmatrix} A_1 (T_a - T_1) \\ A_2 (T_a - T_2) \\ A_3 (T_a - T_3) \end{pmatrix}.$$
So for the time being we model convection as being linear in the temperature difference. This is called [Newton's Law of Cooling](https://en.wikipedia.org/wiki/Newton's_law_of_cooling), and appears to be decent for forced convection.
We completely ignore radiation, since I think its influence is negligible. Using Stefan-Boltzmann law, we can estimate the radiative power over the surface as the blocks to be on the order of $10^2~\text{W}/\text{m}^2$. If we are dealing with metals, we can achieve values of $m c_p \approx 10^3~\text{J}/\text{K}$ (as we choose later on), with masses on the order of $10^0~\text{kg}$, or volumes on the order of $10^{-3}~\text{m}^3$. This suggests a typical surface area of $10^{-2}~\text{m}^2$, so that the net heat transfer due to radiation is roughly $10^0~\text{W}$. Our other heat terms are all $10^2~\text{W}$, so the contribution of radiation is comparatively small. 

## Simulate Data
To get a feel for the dynamics, we will first visualise the evolution of the temperatures in our system. We should already get quite interesting dynamic behaviour, since we have a lot of coupling going on, as well as input heats which can be made arbitrarily complicated.

In [ ]:
Φ(B, ω, t) = B * sin(ω * t)

function LSSM_lump(dT, T, p, t)
    mcp_1, mcp_2, mcp_3, A_1, A_2, A_3, B_1, B_2, B_3, ω_1, ω_2, ω_3, T_a, k12, k23, h_a = p
    # Conduction
    dT[1] = k12 * (T[2] - T[1]) / mcp_1
    dT[2] = (k12 * (T[1] - T[2]) + k23 * (T[3] - T[2])) / mcp_2
    dT[3] = k23 * (T[2] - T[3]) / mcp_3
    # Convection
    dT[1] += h_a * A_1 * (T_a - T[1]) / mcp_1
    dT[2] += h_a * A_2 * (T_a - T[2]) / mcp_2
    dT[3] += h_a * A_3 * (T_a - T[3]) / mcp_3
    # Input
    dT[1] += Φ(B_1, ω_1, t) / mcp_1
    dT[2] += Φ(B_2, ω_2, t) / mcp_2
    dT[3] += Φ(B_3, ω_3, t) / mcp_3
end

Next, we define all the true system parameters, as well as our observations (the measurement error $\sigma$ is assumed to be unknown in this example). The chosen parameter values are not necessarily particularly realistic.

[comment]: # (To reduce warnings from DifferentialEquations.jl, I have to increase the step size. In order to keep the same physics, I multiply the heat capacity $m c_p$, which represents the amount of heat needed to increase the temperature of a block by a certain temperature, by the factor I want to slow it down)

In [ ]:
# Time horizon
sample_size = 100
Δ = 5
time = [Δ * i for i in 0:sample_size]
# Constants of blocks
true_mcp_1 = 2e3
true_mcp_2 = 1.5e3
true_mcp_3 = 2.5e3
true_A_1 = 1.
true_A_2 = 1.
true_A_3 = 1.
# Input heat parameters
true_B_1 = 300.
true_B_2 = -300.
true_B_3 = 150.
true_ω_1 = 3e-2
true_ω_2 = 2.4e-2
true_ω_3 = 3.8e-2
# Temperatures
true_T_a = 290.
T_0 = [330., 270., 310.]
# Unknown constants
true_k12 = 5.
true_k23 = 4.
true_h_a = 2.
true_p = [true_mcp_1, true_mcp_2, true_mcp_3, true_A_1, true_A_2, true_A_3, true_B_1, true_B_2, true_B_3, true_ω_1, true_ω_2, true_ω_3, true_T_a, true_k12, true_k23, true_h_a] # First known, then unknown parameters
# Solve the system numerically using DifferentialEquations.jl
LSSM_dynamics_lump = ODEProblem(LSSM_lump, T_0, (time[1], time[end]), true_p)
true_sol = solve(LSSM_dynamics_lump, Tsit5(); saveat = Δ, verbose = false)
# Get measurement data
true_σ = 1.
y = Array(true_sol) + true_σ * randn(size(Array(true_sol)));

We visualise the true solution as well as the observations:

In [ ]:
plot(true_sol, xlim = (time[1], time[end]), ylim = (260, 340), linecolors = ["red" "blue" "orange"], labels = [L"$T_1$ true" L"$T_2$ true" L"$T_3$ true"], xlabel = L"t~(\textrm{s})", ylabel = L"T~(\textrm{K})", size = (1200, 400), bottommargin = 6mm, leftmargin = 6mm)
scatter!(true_sol.t, y', markercolors = ["red" "blue" "orange"], labels = [L"$T_1$ observed" L"$T_2$ observed" L"$T_3$ observed"])

In [ ]:
time_axis = Vector{String}(undef, sample_size + 1)
time_axis .= ""
steps = 5
step = div(sample_size, steps)
for i ∈ 0:steps
    time_axis[i * step + 1] = string(i * step * Δ)
end
heatmap(Array(true_sol), xticks = (1:(sample_size + 1), time_axis), xlabel = L"t~(\textrm{s})", ylabel = "Block")

## Probabilistic Model
To perform the approximate inferencing, we use Turing.jl. We need to define our model, which includes a bunch of fairly uninformative priors. Since physically it makes sense for $k_{12}, k_{23}, h_a > 0$ (otherwise heat would flow from cold places to warm places...), I decided to give all of them Gamma priors:

In [ ]:
@model function fit_LSSM_lump(data, system, syst_consts)
    # Gamma in Distributions.jl is shape-scale, not shape-rate
    σ ~ Gamma(1e0, 1e0) # E[σ] = 10^0, Var(σ) = 10^0
    k12 ~ Gamma(1e0, 1e0) # E[k] = 10^0, Var(k) = 10^0
    k23 ~ Gamma(1e0, 1e0)
    h_a ~ Gamma(1e0, 1e0) # E[h_a] = 10^0, Var(h_a) = 10^0
    T_a, mcp_1, mcp_2, mcp_3, A_1, A_2, A_3, B_1, B_2, B_3, ω_1, ω_2, ω_3, Δ = syst_consts
    p = [mcp_1, mcp_2, mcp_3, A_1, A_2, A_3, B_1, B_2, B_3, ω_1, ω_2, ω_3, T_a, k12, k23, h_a]
    predicted = solve(system, Tsit5(); p = p, saveat = Δ, verbose = false)

    for i ∈ 1:length(predicted)
        data[:, i] ~ MvNormal(predicted[i], σ^2 * I)
    end
end

## Inference
Finally, we can perform the sampling: 

In [ ]:
model_lump = fit_LSSM_lump(y, LSSM_dynamics_lump, [true_T_a, true_mcp_1, true_mcp_2, true_mcp_3, true_A_1, true_A_2, true_A_3, true_B_1, true_B_2, true_B_3, true_ω_1, true_ω_2, true_ω_3, Δ]);
chain_lump = sample(model_lump, NUTS(0.65), MCMCSerial(), 2500, 3; verbose = false, progress = true)

In order to plot the results, we sample from the Markov chain. For each sampled parameter, we generate a trajectory using our differential equation, which we overlay on the same plot. This shows that for each of the samples the resulting dynamics is not too far off from the real dynamics.

In [ ]:
function sampled_traj_plot(params_samples, solution, data, horizon, time_step, all_params, id_count, system, obs_slices = nothing)
    p_traj = plot(; legend = true, xlim = (horizon[1], horizon[end]), ylim = (260, 340), ylabel = L"T~(\textrm{K})", size = (1200, 400), bottommargin = 6mm, leftmargin = 6mm)
    params_cur = copy(all_params)
    for (i, params_row) ∈ enumerate(eachrow(params_samples))
        params_cur[(end - id_count):end] .= params_row[(end - id_count):end]
        traj_cur = solve(system, Tsit5(); p = params_cur, saveat = time_step)
        if size(Array(traj_cur))[1] == 3
            plot!(p_traj, traj_cur; alpha = 0.05, linecolors = ["red" "blue" "orange"], label = "")
        else
            plot!(p_traj, horizon, Array(traj_cur)[obs_slices, :]'; alpha = 0.05, linecolors = ["red" "blue" "orange"], label = "")
        end
    end
    if obs_slices == nothing
        plot!(p_traj, solution, linecolors = ["red" "blue" "orange"], linewidth = 1, labels = [L"T_1" L"T_2" L"T_3"]) 
    else 
        plot!(p_traj, horizon, Array(solution)[obs_slices, :]', linecolors = ["red" "blue" "orange"], linewidth = 1, labels = [L"T_1" L"T_2" L"T_3"]) 
    end
    scatter!(p_traj, horizon, data', xlabel = L"t~(\textrm{s})", markercolors = ["red" "blue" "orange"], label = "")
    return p_traj
end

In [ ]:
posterior_samples = Array(sample(chain_lump, 300; replace = false))
plot_sol_app = sampled_traj_plot(posterior_samples, true_sol, y, time, Δ, true_p, 2, LSSM_dynamics_lump)
# savefig(plot_sol_app, "Results/Explore/MCMC/state_evolution_small_noise.pdf")

Let's have a look at the (approximate) posteriors of the parameters. We will quantify the quality of the estimates by calculating the MSE of the posteriors with respect to the true value. The bias-variance decomposition makes this an easy computation.

In [ ]:
get_mean(θ::Symbol, chain) = summarize(chain[[θ]]).nt[:mean][1]
get_var(θ::Symbol, chain) = (summarize(chain[[θ]]).nt[:std])[1]^2
sci_not(value, sigdigits = 2) = replace("$(round(value, sigdigits = sigdigits))", r"e(-?\d+)" => s"\\times 10^{\1}")
MSE(true_θ, post_mean, post_var) = (post_mean - true_θ)^2 + post_var

function MSE_string(θ::Symbol, true_θ, chain)
    mean_θ = get_mean(θ, chain)
    var_θ = get_var(θ, chain)
    MSE_θ = MSE(true_θ, mean_θ, var_θ)
    label = latexstring("\\textrm{MSE} = " * sci_not(MSE_θ))
    return label
end

function marg_post_plot(θ::Symbol, true_θ, chain, ymax = nothing)
    if ymax == nothing
        p_θ = density(chain[[θ]], title = L"%$θ", label = "", legend = true)      
    else
        p_θ = density(chain[[θ]], title = L"%$θ", label = "", legend = true, ylim = (0, ymax))
    end
    vline!(p_θ, [true_θ], label = L"True $%$θ$")
    MSE_label = MSE_string(θ, true_θ, chain)
    annotate!(p_θ, (0.05, 0.95), (MSE_label, 12, :left))
    return p_θ
end

In [ ]:
p_σ = marg_post_plot(:σ, true_σ, chain_lump, 12)
p_k12 = marg_post_plot(:k12, true_k12, chain_lump, 8)
p_k23 = marg_post_plot(:k23, true_k23, chain_lump, 5)
p_h_a = marg_post_plot(:h_a, true_h_a, chain_lump, 12)
plot_marg_post = plot(p_σ, p_k12, p_k23, p_h_a, size = (1200, 800), leftmargin = 8mm, bottommargin = 6mm)
# savefig(plot_marg_post, "Results/Explore/MCMC/marg_post_dists_small_noise.pdf")

On this fairly complicated physical model, MCMC appears to perform well. 

Next, we consider how changing the variance of the measurement noise affects the performance of the inference. We start by increasing the variance of the noise by a factor 100.

In [ ]:
# Get measurement data
true_σ = 10.
y = Array(true_sol) + true_σ * randn(size(Array(true_sol)))
model_lump = fit_LSSM_lump(y, LSSM_dynamics_lump, [true_T_a, true_mcp_1, true_mcp_2, true_mcp_3, true_A_1, true_A_2, true_A_3, true_B_1, true_B_2, true_B_3, true_ω_1, true_ω_2, true_ω_3, Δ]);
chain_lump = sample(model_lump, NUTS(0.65), MCMCSerial(), 2500, 3; verbose = false, progress = true)

In [ ]:
posterior_samples = Array(sample(chain_lump, 300; replace = false))
plot_sol_app = sampled_traj_plot(posterior_samples, true_sol, y, time, Δ, true_p, 2, LSSM_dynamics_lump)
# savefig(plot_sol_app, "Results/Explore/MCMC/state_evolution_big_noise.pdf")

In [ ]:
p_σ = marg_post_plot(:σ, true_σ, chain_lump, 1.25)
p_k12 = marg_post_plot(:k12, true_k12, chain_lump, 0.8)
p_k23 = marg_post_plot(:k23, true_k23, chain_lump, 0.6)
p_h_a = marg_post_plot(:h_a, true_h_a, chain_lump, 1.25)
plot_marg_post = plot(p_σ, p_k12, p_k23, p_h_a, size = (1200, 800), leftmargin = 8mm, bottommargin = 6mm)
# savefig(plot_marg_post, "Results/Explore/MCMC/marg_post_dists_big_noise.pdf")

For each parameter, the increase in the variance leads to a roughly proportional increase in the MSE (i.e. the MSE increases by an order $10^2$).

Finally, we test what happens when there is no measurement noise. 

In [ ]:
# Get measurement data
true_σ = 0.
y = Array(true_sol) + true_σ * randn(size(Array(true_sol)))
model_lump = fit_LSSM_lump(y, LSSM_dynamics_lump, [true_T_a, true_mcp_1, true_mcp_2, true_mcp_3, true_A_1, true_A_2, true_A_3, true_B_1, true_B_2, true_B_3, true_ω_1, true_ω_2, true_ω_3, Δ]);
chain_lump = sample(model_lump, NUTS(0.65), MCMCSerial(), 2500, 3; verbose = false, progress = true)

In [ ]:
posterior_samples = Array(sample(chain_lump, 300; replace = false))
plot_sol_app = sampled_traj_plot(posterior_samples, true_sol, y, time, Δ, true_p, 2, LSSM_dynamics_lump)
# savefig(plot_sol_app, "Results/Explore/MCMC/state_evolution_no_noise.pdf")

In [ ]:
p_σ = marg_post_plot(:σ, true_σ, chain_lump, 600)
p_k12 = marg_post_plot(:k12, true_k12, chain_lump, 500)
p_k23 = marg_post_plot(:k23, true_k23, chain_lump, 250)
p_h_a = marg_post_plot(:h_a, true_h_a, chain_lump, 700)
plot_marg_post = plot(p_σ, p_k12, p_k23, p_h_a, size = (1200, 800), leftmargin = 8mm, bottommargin = 6mm)
# savefig(plot_marg_post, "Results/Explore/MCMC/marg_post_dists_no_noise.pdf")

Overall, the inference has performed well. Only the posterior of the measurement noise $\sigma$ is not very good. This might be caused by the fact that I used a Gamma prior, while $\sigma = 0$ is not in the support of the Gamma distribution.  

## Conclusion
From these experiments, it seems like MCMC, implemented by combining Turing.jl with DifferentialEquations.jl, is a good approximate technique for performing Bayesian inference:
- The method is fairly general, which means that it should not be necessary to develop (parts of) software packages;
- The method seems to be fairly insensitive to how informative the priors are, although this might change if we were to introduce some process noise;
- The accuracy (in terms of MSE) of the inference seems to scale with the measurement noise in a roughly linear way.

MCMC is fairly slow however, and so it would not be suitable for online parameter estimation. It would be more suited to occasional calibrations. This is not a problem for my project, however. Consequently, during the rest of this project we will work with MCMC. We continue in the main [Julia Jupyter notebook](sysid-thermal-AR.ipynb).